In [3]:
import pandas as pd
from sqlalchemy import create_engine #pip install sqlalchemy ; ##%pip install SQLAlchemy --quiet
engine = create_engine('sqlite:///Crypto_1h.db')
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import timedelta
import talib

In [ ]:
symbols =pd.read_sql("""SELECT name FROM sqlite_schema WHERE type='table'""",engine)
symbols

In [4]:
class SmaCross(Strategy):
    n1 = 50
    n2 = 100

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)
    
    def next(self):
        price = self.data.Close
        if crossover(self.sma1, self.sma2):
            sl = price - price *0.03
            tp = price + price *0.04
            self.buy(sl = sl, tp = tp)

In [17]:
class EmaCross(Strategy): #v1
    emaF=14
    emaS=63
    
    def init(self):
        close = self.data.Close
        self.emaFast = self.I(talib.EMA, close, self.emaF)
        self.emaSlow = self.I(talib.EMA, close, self.emaS)
        
    def next(self):
        price = self.data.Close
        if crossover(self.emaFast,self.emaSlow):
            sl = price - price * 0.05
            tp = price + price * 0.04
            self.buy(sl = sl, tp = tp)
        elif crossover(self.emaSlow,self.emaFast):
            sl = price + price * 0.04
            tp = price - price * 0.05
            self.sell(sl = sl, tp = tp)

In [44]:
class EmaCross(Strategy): #v2
    emaF=14
    emaS=63
    
    def init(self):
        close = self.data.Close
        self.emaFast = self.I(talib.EMA, close, self.emaF)
        self.emaSlow = self.I(talib.EMA, close, self.emaS)
        
    def next(self):
        price = self.data.Close
        if crossover(self.emaFast,self.emaSlow):
            self.buy()
        elif crossover(self.emaSlow,self.emaFast):
            self.sell()

In [45]:
def resampleOHLC(df, interval):
    df = df.resample(interval).agg({'Open':'first','High':'max','Low':'min','Close':'last'})
    return df

In [46]:
def BT(interval):

    returns = []
    
    for symbol in symbols.name:
        qry = f"""SELECT * FROM '{symbol}'
        WHERE Date < '{pd.to_datetime('today') - timedelta(days=30)}'"""#80days-30
        df = pd.read_sql(qry, engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df, interval)
        bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
        output = bt.run()
        returns.append(output['Return [%]'])
        
    frame = pd.DataFrame(returns, index=symbols.name, columns=['ret'])
    top_5 = frame.nlargest(5,'ret')
    return top_5, interval

In [47]:
top_5, interval = BT('1h')  #Todat - 80 until today - 30

C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\4073476979.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\4073476979.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\4073476979.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-

In [48]:
top_5

,ret
name,
BAKEUSDT,94.885537
IOTAUSDT,26.018279
ETCUSDT,21.445185
MATICUSDT,12.111940
SOLUSDT,7.023412


In [55]:
def validate(top_5, interval):

    returns = []
    for symbol in top_5.index:
        qry = f"""SELECT * FROM '{symbol}'
        WHERE Date > '{pd.to_datetime('today') - timedelta(days=10)}'"""#last 10 days
        df = pd.read_sql(qry, engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df, interval)
        bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
        output = bt.run()
        returns.append(output['Return [%]'])
    outcome = pd.DataFrame(returns, index=top_5.index, columns=['ret'])
    return outcome
    


In [56]:
validate(top_5, interval)

,ret
name,
BAKEUSDT,12.608540
IOTAUSDT,6.940620
ETCUSDT,-0.605251
MATICUSDT,-3.345868
SOLUSDT,4.162005


In [42]:
def run(interval):
    bt_result, interval = BT(interval)
    valid = validate(bt_result,interval)
    return valid.ret.mean()

In [36]:
#Test
#run('15min')

C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134

-5.940460294100019

In [43]:
#Test 15, 30, 45, 60 [min]
int_scale = 'min'

for i in range (15,75,15):
    print('return for '+str(i) +int_scale)
    print(run(str(i)+ int_scale))

return for 15min


C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134

-5.940460294100019
return for 30min


C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134

-18.335951056000038
return for 45min


C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134

-12.283164443700006
return for 60min


C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, EmaCross, exclusive_orders=True, cash=1000, commission=0.0015)
C:\Users\JulioIsaacMR\AppData\Local\Temp\ipykernel_15824\112464906.py:11: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134

-28.5682427726
